In [ ]:
import numpy as np
import fsspec
import json
import h5py
from fsspec.implementations.cached import CachingFileSystem

# nwb_object_id = 'c86cdfba-e1af-45a7-8dfd-d243adc20ced'
nwb_object_id = '66443b03-5266-48ba-af4b-f2fe4026acff'
s3_url = f'https://dandiarchive.s3.amazonaws.com/blobs/{nwb_object_id[:3]}/{nwb_object_id[3:6]}/{nwb_object_id}'

fs = CachingFileSystem(
    fs=fsspec.filesystem("http"),
    cache_storage="nwb-cache",  # Local folder for the cache
)

print('Opening file for lazy reading...')
f = fs.open(s3_url, "rb")
file = h5py.File(f)

In [ ]:
specifications = file['specifications']

In [ ]:
specifications['hdmf-common']['1.5.0'].keys()

In [ ]:
inheritance1 = {}
X1 = specifications['core']['2.3.0']
X2 = specifications['hdmf-common']['1.5.0']
for X in [X1, X2]:
    for k, v in X.items():
        if k == 'namespace':
            continue
        print('====' + k)
        js = v[()]
        spec = json.loads(js.decode('utf-8'))
        if 'datasets' in spec:
            for ds in spec['datasets']:
                if 'neurodata_type_def' in ds and 'neurodata_type_inc' in ds:
                    print(ds['neurodata_type_def'], ds['neurodata_type_inc'])
                    inheritance1[ds['neurodata_type_def']] = ds['neurodata_type_inc']
        if 'groups' in spec:
            for gr in spec['groups']:
                if 'neurodata_type_def' in gr and 'neurodata_type_inc' in gr:
                    print(gr['neurodata_type_def'], gr['neurodata_type_inc'])
                    inheritance1[gr['neurodata_type_def']] = gr['neurodata_type_inc']

In [ ]:
print(json.dumps(inheritance1, indent=4))

In [ ]:
inheritance2 = {}
for k, v in inheritance1.items():
    inheritance2[k] = []
    v2 = v
    while True:
        inheritance2[k].append(v2)
        if v2 not in inheritance1:
            break
        v2 = inheritance1[v2]
for k, v in inheritance2.items():
    print(k, v)

In [ ]:
print(json.dumps(inheritance2, indent=4))